In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt, wkb 
from shapely.geometry import LineString
import json
import getpass
import requests
from requests.auth import HTTPBasicAuth

import STARTHER
import bomst2datafangst
import nvdbapiv3

Legger NVDB api til søkestien


# Input fra Turid 

Jeg hadde som dere kjenner til en bestilling på kart over trafikkstasjonene i hele landet i april, og oppdaget raskt at det som lå i NVDB var svært mangelfullt og feil.
Da var det like greit å gjøre en jobb på en slik at datasettet i NVDB kan forbedres, tenkte jeg. 
Så, vedlagte fil inneholder ajourførte data ihht Vegvesen.no sin oversikt, og skal være grei å få inn i NVDB, trur jeg. Man ser hvor det lå data inne fra før, men de fleste av disse har endret geometri og egenskaper. 
Følgende er utført:
  * Hentet ut det som fantes fra 638_Trafikkstsjon i NVDB
  * Lagt inn manglende stasjoner, og flyttet eksisterende geometri til riktig adresse.
  * Endret/lagt inn alle minimumsegenskapene «Navn», «link til infosider» og «Geometri_ punkt» 

Da overlater jeg stafettpinnene til dere data-INN folk. 

Hilsen Turid



# Gjøremål

### For nye objekter: 

 * Turid har plassert objektene geografisk på rett sted, men ikke plassert på vegnettet 
   * **=> Sendte nye objekter til DATAFANGST**
   * Har lagret de nye trafikkstasjonene til PROD. Gjorde en del QA i Datafangst: Feil geometri på Kirkenes (lå i Harstad Havn) og Kvenvikmoen (lå i Kirkenes, hører hjemme i Alta). Fiksa med geometriredigering i Datafangst
   
### For eksisterende objekter 
  * Er egengeometrien oppdatert? 
    * Sjekk avstand ny egengeometri vs gammel egengeometri
  * Tilfører egengeometrien reell informasjon? 
    * Bør være en viss avstand mellom senterlinje geometri - egengeometri. 
    * I NVDB er det masse egengeometrier som ligger på senterlinja, da bør vi fjerne egengeometrien? 
  * Oppdater egenskapsverdier med ny informasjon, inklusive de egengeometriene som er forbedret 
  
Hm, Hønefoss Trafikkstasjon er dublett, hvem skal vi beholde? 
  * 894209984 har stedfesting 0.82355361@181254 MED V, egengeometri POINT (237387.529872853 6685601.37970294)
  * 894210102 har stedfesting 0.82350646@181254 MED V, egengeometri POINT (237389.526595896 6685601.77904755)

Turid har videreført 894209984, jeg slettet 894210102. 



  
  # Leser Turid sitt datasett, sender nye objekt til Datafangst 

In [2]:
traf = gpd.read_file( '../wip_tungedata/Trafikkstasjoner_ajour_2022/Trafikkstasjoner.gdb')

In [3]:
traf['geometry_turid'] =   traf['geometry'].apply(       lambda x :  wkb.loads( wkb.dumps( x, output_dimension=2 )))
traf['geometry_egen']  = traf['Geometri__punkt'].apply(  lambda x :  wkb.loads( wkb.dumps( wkt.loads( x ) , output_dimension=2 )))

In [4]:
def laglinje( row ): 
    return LineString( [ row['geometry_turid'], row['geometry_egen'] ] )

traf['geometry'] = traf.apply( laglinje, axis = 1)

/home/jan/anaconda3/envs/jupyterlabAug2022/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
traf['avstand'] = traf['geometry'].length

In [9]:
traf[ traf['avstand'] > 3]['avstand']

6       3.781648
9       4.400422
31     21.478722
47    291.537661
Name: avstand, dtype: float64

In [6]:
traf[ traf['Navn'].str.contains('Hønefoss')]

,OBJECTID_1,Navn,Link_til_infosider,Tilleggsinformasjon,Geometri__punkt,nvdb_id,nvdb_metadata_sist_modifisert,geometry,geometry_turid,geometry_egen,avstand
47,48.0,Hønefoss trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,,POINT (237387.529872853 6685601.37970294),894209984.0,2021-11-17T09:44:05,"LINESTRING (237143.020 6685442.605, 237387.530...",POINT (237143.020 6685442.605),POINT (237387.529872853 6685601.37970294),291.537661


In [7]:
traf[ traf['nvdb_id'] < 1]['avstand'].describe()

count    30.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: avstand, dtype: float64

###  Lager ny geodataframe med kun de nye stasjonene 

In [8]:
nyestasjoner = traf[ traf['nvdb_id'] < 1][['Navn', 'Link_til_infosider', 'geometry_turid']].copy()
nyestasjoner.rename( columns={ 'geometry_turid' : 'geometry', 'Link_til_infosider' : 'Link til infosider'}, inplace=True )
nyestasjoner = gpd.GeoDataFrame( nyestasjoner, geometry='geometry', crs=5973)
nyestasjoner.explore(  marker_kwds={'radius' : 5} )

In [9]:
nyestasjoner = nyestasjoner.to_crs( 'epsg:4326')

In [33]:
nyestasjoner.reset_index( inplace=True)

In [34]:
myGjson = bomst2datafangst.gdf2geojson( nyestasjoner, objektTypeId=638, comment='Fra Turid Vongraven')

In [35]:
with open( 'trafikkstasjoner.geojson', 'w') as f:
    json.dump( myGjson, f)

In [23]:
kontrakt = '4f874789-e209-43a0-8ba3-745e2a58c4e2'
dfApi = 'https://datafangst.test.vegvesen.no/api/v1/contract/'
url = dfApi + kontrakt +  '/featurecollection'
username = 'jajens'
headers = headers = { 'Content-Type' : 'application/geo+json', 'Accept' : 'application/json' }

In [17]:
# pw = getpass.getpass( username+"'s TESTPROD passord")

jajens's TESTPROD passord········


In [45]:
# r = requests.post( url, headers=headers, auth=HTTPBasicAuth(username, pw), data=json.dumps(  myGjson ) )

In [46]:
r

<Response [401]>

### Sender til datafangst PROD

In [52]:
kontraktPROD = '2edaf0ac-116d-4320-b3d7-6763406597fb'
dfApiPROD = 'https://datafangst.vegvesen.no/api/v1/contract/'
urlPROD = dfApiPROD + kontraktPROD +  '/featurecollection'
username = 'jajens'
headers = headers = { 'Content-Type' : 'application/geo+json', 'Accept' : 'application/json' }

In [49]:
# pwProd = getpass.getpass( username+"'s PROD password:")

jajens's PROD password:········


In [53]:
# r = requests.post( urlPROD, headers=headers, auth=HTTPBasicAuth(username, pwProd), data=json.dumps(  myGjson ) )

In [54]:
r

<Response [202]>

Har lagret de nye trafikkstasjonene til PROD. Gjorde en del QA i Datafangst: Feil geometri på Kirkenes (lå i Harstad Havn) og Kvenvikmoen (lå i Kirkenes, hører hjemme i Alta). Fiksa med geometriredigering i Datafangst

# Sjekker Turids data mot eksisterende NVDB data

In [76]:
traf = gpd.read_file( '../wip_tungedata/Trafikkstasjoner_ajour_2022/Trafikkstasjoner.gdb') 
# Leser på ny, vi har gjort så mye rart med datene
# Kun eksisterende objekt, og v i tar ikke med alle egenskaper
# Dropper Tilleggsinformasjon, for der lå det ingenting 
traf = traf[ traf['nvdb_id'] > 1][['Navn', 'Link_til_infosider', 'Geometri__punkt', 'nvdb_id', 'geometry' ]].copy()
traf.rename( columns={'nvdb_id' : 'nvdbId', 'Link_til_infosider' : 'Link til infosider', 
                      'Geometri__punkt' : 'Geometri, punkt' }, inplace=True  )
traf['nvdbId'] = traf['nvdbId'].astype(int)

In [64]:
sok = nvdbapiv3.nvdbFagdata( 638)
nvdbtraf = pd.DataFrame( sok.to_records(relasjoner=False, geometri=True))

In [66]:
nvdbtraf.rename( columns={ 'Geometri, punkt' : 'nvdbEgengeometri',
       'Navn' : 'nvdbNavn',  'Link til infosider' : 'nvdbLink til infosider', 
       'geometri' : 'vegnettsgeometri'  }, inplace=True)

In [70]:
print( traf.columns)
print( nvdbtraf.columns)

Index(['Navn', 'Link til infosider', 'Geometri, punkt', 'nvdbId', 'geometry'], dtype='object')
Index(['objekttype', 'nvdbId', 'versjon', 'startdato', 'nvdbEgengeometri',
       'nvdbNavn', 'nvdbLink til infosider', 'veglenkesekvensid', 'detaljnivå',
       'typeVeg', 'kommune', 'fylke', 'vref', 'veglenkeType', 'vegkategori',
       'fase', 'vegnummer', 'relativPosisjon', 'adskilte_lop',
       'trafikantgruppe', 'vegnettsgeometri', 'Tilleggsinformasjon'],
      dtype='object')


In [77]:
joined = pd.merge( traf, nvdbtraf, on='nvdbId', how='inner')

In [73]:
# Lager ny geometri i 2D for 

Index(['Navn', 'Link til infosider', 'Geometri, punkt', 'nvdbId', 'geometry',
       'objekttype', 'versjon', 'startdato', 'nvdbEgengeometri', 'nvdbNavn',
       'nvdbLink til infosider', 'veglenkesekvensid', 'detaljnivå', 'typeVeg',
       'kommune', 'fylke', 'vref', 'veglenkeType', 'vegkategori', 'fase',
       'vegnummer', 'relativPosisjon', 'adskilte_lop', 'trafikantgruppe',
       'vegnettsgeometri', 'Tilleggsinformasjon'],
      dtype='object')

In [82]:
joined['geomNyEgengeom'] = joined['Geometri, punkt'].apply(  lambda x :  wkb.loads( wkb.dumps( wkt.loads( x ) , output_dimension=2 )))
joined['geomVegnett'] = joined['vegnettsgeometri'].apply(  lambda x :  wkb.loads( wkb.dumps( wkt.loads( x ) , output_dimension=2 )))
joined['geomNvdbEgen'] = joined['nvdbEgengeometri'].apply(  lambda x :  wkb.loads( wkb.dumps( wkt.loads( x ) , output_dimension=2 )))

,Navn,Link til infosider,"Geometri, punkt",nvdbId,geometry,objekttype,versjon,startdato,nvdbEgengeometri,nvdbNavn,...,fase,vegnummer,relativPosisjon,adskilte_lop,trafikantgruppe,vegnettsgeometri,Tilleggsinformasjon,geomNyEgengeom,geomVegnett,geomNvdbEgen
0,Kristiansund trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (136093.024505357 7018299.92945552),154788377,POINT Z (136093.025 7018299.929 0.000),638,1,2008-01-15,POINT (136093.024505357 7018299.92945552),Kristiansund trafikkstasjon,...,V,70,0.173011,Nei,K,POINT Z(136086.621 7018355.692 24.391),NaN,POINT (136093.024505357 7018299.92945552),POINT (136086.621 7018355.692),POINT (136093.024505357 7018299.92945552)
1,Sunndalsøra trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (170680.599356925 6965811.34221204),154788378,POINT Z (170680.599 6965811.342 0.000),638,1,2008-01-15,POINT (170680.599356925 6965811.34221204),Sunndalsøra trafikkstasjon,...,V,62,0.891078,Nei,K,POINT Z(170672.409 6965834.14 6.492),NaN,POINT (170680.599356925 6965811.34221204),POINT (170672.409 6965834.14),POINT (170680.599356925 6965811.34221204)
2,Ørsta trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (36963.8891099699 6926371.65641883),154788379,POINT Z (36963.889 6926371.656 0.000),638,1,2008-01-15,POINT (36963.8891099699 6926371.65641883),Ørsta trafikkstasjon,...,V,39,0.671029,Nei,K,POINT Z(36908.068 6926561.783 64.623),NaN,POINT (36963.8891099699 6926371.65641883),POINT (36908.068 6926561.783),POINT (36963.8891099699 6926371.65641883)
3,Steinkjer trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (329088.977204681 7104603.41052957),154788381,POINT Z (329088.977 7104603.411 0.000),638,1,2008-01-15,POINT (329088.977204681 7104603.41052957),Steinkjer trafikkstasjon,...,V,763,0.122118,Nei,K,POINT Z(329140.357 7104550.973 16.118),NaN,POINT (329088.977204681 7104603.41052957),POINT (329140.357 7104550.973),POINT (329088.977204681 7104603.41052957)
4,Levanger trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (317451.618690283 7073998.98935016),154788382,POINT Z (317451.619 7073998.989 0.000),638,1,2008-01-15,POINT (317451.618690283 7073998.98935016),Levanger servicekontor,...,V,6874,0.034206,Nei,K,POINT Z(317394.911 7074051.443 6.885),NaN,POINT (317451.618690283 7073998.98935016),POINT (317394.911 7074051.443),POINT (317451.618690283 7073998.98935016)
5,Namsos trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (333295.042762124 7153895.50235787),154788383,POINT Z (333295.043 7153895.502 0.000),638,1,2008-01-15,POINT (333295.042762124 7153895.50235787),Namsos trafikkstasjon,...,V,17,0.000000,Nei,K,POINT Z(333756.4 7153914.5 14.61),NaN,POINT (333295.042762124 7153895.50235787),POINT (333756.4 7153914.5),POINT (333295.042762124 7153895.50235787)
6,Molde trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (106276.579150047 6983033.51386789),154788384,POINT Z (106275.987 6983029.779 0.000),638,1,2008-01-15,POINT (106276.579150047 6983033.51386789),Molde trafikkstasjon,...,V,538,0.962214,Nei,K,POINT Z(106231.303 6983073.051 109.589),NaN,POINT (106276.579150047 6983033.51386789),POINT (106231.303 6983073.051),POINT (106276.579150047 6983033.51386789)
7,Brekstad trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (236292.876185029 7074034.37423679),154788385,POINT Z (236292.876 7074034.374 0.000),638,1,2008-01-16,POINT (236292.876185029 7074034.37423679),Brekstad trafikkstasjon,...,V,710,0.108707,Nei,K,POINT Z(236246.692 7074060.898 3.42),NaN,POINT (236292.876185029 7074034.37423679),POINT (236246.692 7074060.898),POINT (236292.876185029 7074034.37423679)
8,Ålesund trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,POINT (55591.2545423125 6956702.94670949),154788388,POINT Z (55591.254 6956702.947 0.000),638,1,2008-01-15,POINT (55591.2545423125 6956702.94670949),Ålesund trafikkstasjon,...,V,5946,0.982229,Nei,K,POINT Z(55736.656 6956592.775

In [58]:
nvdbtraf['geometry'] = nvdbtraf['geometri'].apply( wkt.loads )
nvdbtraf = gpd.GeoDataFrame( nvdbtraf, geometry='geometry', crs=5973)

In [63]:
nvdbtraf.to_file( 'trafikkstasjoner.gpkg', layer='nvdb-trafikkstasjoner', crs=5973)

In [120]:
nvdbtraf['vegkategori'].value_counts()

F    27
E    14
R    10
Name: vegkategori, dtype: int64

In [121]:
nvdbtraf[ nvdbtraf['Navn'].str.contains('Hønefoss')]

,objekttype,nvdbId,versjon,startdato,"Geometri, punkt",Navn,Link til infosider,veglenkesekvensid,detaljnivå,typeVeg,...,vref,veglenkeType,vegkategori,fase,vegnummer,relativPosisjon,adskilte_lop,trafikantgruppe,geometri,geometry
47,638,894209984,1,2018-08-28,POINT (237387.529872853 6685601.37970294),Hønefoss trafikkstasjon,https://www.vegvesen.no/om+statens+vegvesen/ko...,181254,Vegtrase og kjørebane,Enkel bilveg,...,EV16 S47D1 m3409,HOVED,E,V,16,0.823554,Nei,K,POINT Z(237389.407 6685602.17 202.825),POINT Z (237389.407 6685602.170 202.825)
50,638,894210102,1,2018-08-28,POINT (237389.526595896 6685601.77904755),Hønefoss trafikkstasjon,https://www.vegvesen.no/om+statens+vegvesen/ko...,181254,Vegtrase og kjørebane,Enkel bilveg,...,EV16 S47D1 m3409,HOVED,E,V,16,0.823506,Nei,K,POINT Z(237389.565 6685601.795 202.817),POINT Z (237389.565 6685601.795 202.817)


In [64]:
nvdbtraf.explore( marker_kwds={'radius' : 5} )

In [60]:
nvdbtraf.dtypes

objekttype               int64
nvdbId                   int64
versjon                  int64
startdato               object
Geometri, punkt         object
Navn                    object
Link til infosider      object
veglenkesekvensid        int64
detaljnivå              object
typeVeg                 object
kommune                  int64
fylke                    int64
vref                    object
veglenkeType            object
vegkategori             object
fase                    object
vegnummer                int64
relativPosisjon        float64
adskilte_lop            object
trafikantgruppe         object
geometri                object
geometry              geometry
dtype: object

In [43]:
traf[ traf['avstand'] > 3]

,OBJECTID_1,Navn,Link_til_infosider,Tilleggsinformasjon,Geometri__punkt,nvdb_id,nvdb_metadata_sist_modifisert,geometry,geometry_turid,geometry_egen,avstand
6,7.0,Molde trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,,POINT (106276.579150047 6983033.51386789),154788384.0,2020-10-08T14:55:45,"LINESTRING (106275.987 6983029.779, 106276.579...",POINT (106275.987 6983029.779),POINT (106276.579150047 6983033.51386789),3.781648
9,10.0,Trondheim trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,,POINT (270413.806580184 7038241.26675203),154788389.0,2022-02-24T10:53:26,"LINESTRING (270417.872 7038242.951, 270413.807...",POINT (270417.872 7038242.951),POINT (270413.806580184 7038241.26675203),4.400422
31,32.0,Voss trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,,POINT Z(34816.8643537077 6752689.57057003 94.483),480800283.0,2021-10-05T13:38:29,"LINESTRING (34836.168 6752698.989, 34816.864 6...",POINT (34836.168 6752698.989),POINT (34816.8643537077 6752689.57057003),21.478722
47,48.0,Hønefoss trafikkstasjon,https://www.vegvesen.no/om-oss/kontakt-oss/tra...,,POINT (237387.529872853 6685601.37970294),894209984.0,2021-11-17T09:44:05,"LINESTRING (237143.020 6685442.605, 237387.530...",POINT (237143.020 6685442.605),POINT (237387.529872853 6685601.37970294),291.537661


In [61]:
traf[['Navn', 'nvdb_id', 'avstand', 'geometry'  ]].explore(style_kwds= {'weigth' : 10, 'color' : 'red' } )

In [72]:
nyeobj = traf[ traf['nvdb_id'] == 0].copy()


In [75]:
nyeobj['geometry'] = nyeobj['geometry_turid']

In [76]:
nyeobj = gpd.GeoDataFrame( nyeobj, geometry='geometry', crs=5973)

In [87]:
nyeobj.columns

Index(['OBJECTID_1', 'Navn', 'Link_til_infosider', 'Tilleggsinformasjon',
       'Geometri__punkt', 'nvdb_id', 'nvdb_metadata_sist_modifisert',
       'geometry', 'geometry_turid', 'geometry_egen', 'avstand'],
      dtype='object')

In [91]:
nyeobj[ ['Navn', 'Link_til_infosider', 'Tilleggsinformasjon', 'geometry' ]].explore(  marker_kwds={'radius' : 5} )